# Representativity - Ecoregion coverage 1.2

### Import libraries and set up environment

In [6]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [7]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [40]:
# Union the dissolved MPAs with the ecoregions

in_feature_1 = "WDPA_MPA_dis"
in_feature_2 = "MEco_mol_Carib"
out_feature = "\\MPA_dis_Eco_union_Rep1_2"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.Union(
    in_features=[in_feature_1,in_feature_2],
    out_feature_class= output_gdb + out_feature,
    join_attributes="ALL",
    cluster_tolerance=None,
    gaps="GAPS"
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\MPA_dis_Eco_union_Rep1_2'>

In [42]:
# Repair geoemtry on union, calculate area, and export to excel

# Set a folder to store the output tables
table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Representativity"

in_feature = "MPA_dis_Eco_union_Rep1_2"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def repair_geo_area_export():
    #repair geometry
    arcpy.management.RepairGeometry(
        in_features= in_feature,
        delete_null="DELETE_NULL",
        validation_method="OGC"
    )
            
    # insert "area" field (double) and calculate area in km2 in Mollweide
    arcpy.management.CalculateGeometryAttributes(
        in_features= in_feature,
        geometry_property="area AREA",
        length_unit="",
        area_unit="SQUARE_KILOMETERS",
        coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
        coordinate_format="SAME_AS_INPUT"
    )
    
    # Export tables to excel
    # Not sure why I had to use the following two lines instead of just using the TableToExcel expression, but I did
    excel_filename = in_feature + ".xlsx"
    excel_filepath = os.path.join(table_outputs_folder, excel_filename)  # Build the full file path

    arcpy.conversion.TableToExcel(
        Input_Table= in_feature,
        Output_Excel_File= excel_filepath,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )
    
repair_geo_area_export()


In [8]:
# Convert the output features to a Pandas DataFrame
df_MPA_union = pd.DataFrame.spatial.from_featureclass("MPA_dis_Eco_union_Rep1_2")


In [9]:
# Display the DataFrame
df_MPA_union.head()

,OBJECTID,FID_WDPA_MPA_dis,FID_MEco_mol_Carib,ECOREGION,REALM,PROVINC,TYPE,BIOME,MEOW_PPOW,area,SHAPE
0,1,108,-1,,,,,,,5.805127e-01,"{""hasZ"": true, ""rings"": [[[-7600501.0242, 2949..."
1,2,-1,1,,Atlantic Warm Water,Equatorial Atlantic,PPOW,Equatorial,Equatorial Atlantic,5.358515e+05,"{""hasZ"": true, ""rings"": [[[-6011097.5789, 2060..."
2,3,-1,2,,Atlantic Warm Water,Gulf Stream,PPOW,Boundary - western,Gulf Stream,6.259979e+04,"{""hasZ"": true, ""rings"": [[[-7164904.4662999995..."
3,4,-1,3,,Atlantic Warm Water,Inter American Seas,PPOW,Semi-enclosed sea,Inter American Seas,3.144845e+06,"{""hasZ"": true, ""rings"": [[[-7893280.767999999,..."
4,5,-1,4,,Atlantic Warm Water,North Atlantic Transitional,PPOW,Transitional,North Atlantic Transitional,8.884828e+04,"{""hasZ"": true, ""rings"": [[[-6910283.536499999,..."


In [10]:
# Make pivot tables
pivot_MPA_ecoregion = df_MPA_union.pivot_table(
     index = "MEOW_PPOW",
     values = "area",
     aggfunc = "sum"
).reset_index()
pivot_MPA_ecoregion.columns = ["MEOW_PPOW", "Total_area"]
#print(pivot_MPA_ecoregion)

pivot_MPA = df_MPA_union[df_MPA_union["FID_WDPA_MPA_dis"] != -1].pivot_table(
    index = "MEOW_PPOW",
    values = "area",
    aggfunc = "sum"
).reset_index()
pivot_MPA.columns = ["MEOW_PPOW", "MPA_area"]
#print(pivot_MPA)

# Merge the pivot tables into a single pivot table. The merge should be based on MEOW_PPOW
ecoregion_coverage = pd.merge(pivot_MPA_ecoregion, pivot_MPA, on="MEOW_PPOW", how="left", validate = "1:1")
#print(ecoregion_coverage)

#add column for the percent MPA coverage per ecoregion
ecoregion_coverage.insert(3, "%MPA_coverage", ecoregion_coverage["MPA_area"] / ecoregion_coverage["Total_area"] * 100)
#print(ecoregion_coverage)

#Replace NaN with 0
ecoregion_coverage = ecoregion_coverage.fillna(0)
print(ecoregion_coverage)

                      MEOW_PPOW    Total_area       MPA_area  %MPA_coverage
0                                5.805127e-01       0.580513     100.000000
1                      Amazonia  8.777461e+01       1.814588       2.067326
2                      Bahamian  1.640552e+05   21895.315308      13.346309
3                    Carolinian  2.660853e+04     277.478896       1.042820
4             Eastern Caribbean  4.470849e+04   14885.648037      33.294901
5           Equatorial Atlantic  6.350137e+05   99162.225160      15.615761
6                     Floridian  1.286773e+05   16363.335705      12.716567
7              Greater Antilles  1.273817e+05   27650.623983      21.706909
8                       Guianan  2.012123e+05    3996.976744       1.986447
9                   Gulf Stream  6.489038e+04    2290.596446       3.529947
10          Inter American Seas  3.339226e+06  194380.567290       5.821127
11  North Atlantic Transitional  8.884828e+04       0.000000       0.000000
12  North Ce

In [11]:
#Export to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Representativity"
table = ecoregion_coverage
output_name = "MPA_dis_Eco_union_Rep1_2_results.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))

Note that there is a small gap in the ecoregion layer that shows up as a blank row in the results excel. I think it can be safely ignored.